In [22]:
import os
import json
import urllib
import multiprocessing
import pandas as pd
import sys
import time
import json
import re

DRIVER_PATH = '/Users/ShinAhnjae/Desktop/3-2/DataJournalism/drivers'
sys.path.append(DRIVER_PATH)

from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime
from functools import reduce, lru_cache
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [24]:
kw_df = pd.read_csv('/Users/ShinAhnjae/Desktop/3-2/DataJournalism/project/datajournalism-project/data/gendata/keyword/list.csv')

In [25]:
kw_df.keyword.unique()

array(['크리스마스 특선영화', '크리스마스 캐롤 모음 자동재생', '크리스마스 택배', '크리스마스 영화',
       '크리스마스 캐롤', '크리스마스 데이트', '크리스마스', '크리스마스 이브', '크리스마스 카드', '크리스마스섬',
       '크리스마스에 눈이 올까요', '크리스마스 동영상', '크리스마스 선물', '크리스마스니까', '크리스마스섬땃쥐',
       '크리스마스 증후군', '크리스마스 축제', '크리스마스 캐롤 모음', '크리스마스 음식', '크리스마스 이미지',
       '크리스마스의 악몽', '크리스마스 로즈', '크리스마스 전날의 악몽', '크리스마스 가볼만한곳', '크리스마스 병원',
       '크리스마스에는 축복을', '크리스마스 트리'], dtype=object)

In [26]:
kw_interest = kw_df.query('keyword in ["크리스마스 축제", "크리스마스 가볼만한곳", "크리스마스 데이트"]')

In [27]:
kw_interest

,rank,keyword,date
171,10.0,크리스마스 데이트,2012-12-24 12:50:00
178,8.0,크리스마스 데이트,2012-12-24 11:40:00
181,9.0,크리스마스 데이트,2012-12-24 11:35:00
186,9.0,크리스마스 데이트,2012-12-24 11:15:00
189,10.0,크리스마스 데이트,2012-12-24 11:10:00
212,9.0,크리스마스 데이트,2012-12-24 02:05:00
213,10.0,크리스마스 데이트,2012-12-24 02:00:00
216,10.0,크리스마스 데이트,2012-12-24 01:30:00
221,8.0,크리스마스 데이트,2012-12-24 01:00:00
222,6.0,크리스마스 데이트,2012-12-24 00:55:00
